In [1]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [28]:
from gspx.utils.display import plot_graph
from gspx.qgsp import QMatrix
from gspx.utils.quaternion_matrix import explode_quaternions, implode_quaternions
import numpy as np

coords = np.array([
    [0, 1],
    [-1, 0],
    [0, -1],
    [1, 0],
    [0, 0],
])

In [3]:
M = np.zeros((5, 5, 4))

idx = np.array([
    [0, 4],
    [1, 4],
    [2, 4],
    [3, 4],
    [3, 0]
])

In [4]:
Aq = QMatrix.from_matrix(implode_quaternions(M))

In [25]:
# from pyquaternion import Quaternion
# from tqdm import tqdm

# Ne = 5

# for i_ in tqdm(range(1000)):
#     rnd = np.random.RandomState(seed=i_)
#     entries = rnd.randint(10, size=(Ne, 4))

#     for n, i in enumerate(idx):
#         Aq.matrix[i[0], i[1]] = Quaternion(entries[n])

#     Aq = Aq + Aq.conjugate().transpose()

#     eigq, Vq = Aq.eigendecompose(hermitian_gso=True)

#     diff = (
#         np.eye(5) -
#         np.round((Vq * Vq.conjugate().transpose()).abs(), decimals=5)).sum()
#     if diff > 0:
#         print(i_)
#         break

In [36]:
from pyquaternion import Quaternion
from tqdm import tqdm

Ne = 5

rnd = np.random.RandomState(seed=42)
entries = rnd.randint(10, size=(Ne, 4))

for n, i in enumerate(idx):
    Aq.matrix[i[0], i[1]] = Quaternion(entries[n])

Aq = Aq + Aq.conjugate().transpose()

eigq, Vq = Aq.eigendecompose(hermitian_gso=True)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

A_ = Aq.abs()
xi, yi = np.where(A_)
edgelist = [
    (xi[n], yi[n], {'weight': A_[xi[n], yi[n]]})
    for n in range(len(xi))
]

g = nx.DiGraph()
g.add_edges_from(edgelist)
plot_graph(
    g, coords, figsize=(3, 3), node_size=60,
    edge_color=(0.8, 0.8, 0.8, 0.6))
fig = plt.gcf()
plt.savefig('/home/gbr/gspx/degenerate_qgft_graph.pdf', dpi=300)

In [32]:
from gspx.utils.quaternion_matrix import explode_quaternions, implode_quaternions
arr = explode_quaternions(Vq.matrix)
arrq = implode_quaternions(np.round(arr, decimals=2))

In [34]:
for row in arrq:
    print([str(q) for q in row[:3]])

['0.400 +0.000i +0.000j +0.000k', '0.610 +0.000i +0.060j +0.040k', '0.000 +0.000i +0.000j +0.000k']
['0.300 -0.100i -0.090j -0.100k', '-0.130 +0.160i +0.140j +0.170k', '-0.300 +0.060i -0.160j +0.610k']
['0.290 -0.100i -0.090j -0.100k', '-0.130 +0.160i +0.130j +0.170k', '0.300 -0.060i +0.170j -0.630k']
['0.190 -0.210i -0.190j -0.200k', '-0.440 -0.220i -0.260j -0.270k', '0.000 +0.000i +0.000j +0.000k']
['0.000 +0.390i +0.380j +0.400k', '-0.140 -0.100i -0.110j -0.130k', '0.000 +0.000i +0.000j +0.000k']


In [35]:
for row in arrq:
    print([str(q) for q in row[3:]])

['-0.410 +0.000i -0.270j -0.120k', '0.460 +0.000i -0.030j -0.040k']
['0.150 +0.090i +0.290j +0.320k', '0.280 +0.050i +0.040j +0.020k']
['0.150 +0.090i +0.280j +0.310k', '0.280 +0.050i +0.040j +0.020k']
['0.140 -0.160i -0.210j -0.410k', '0.400 +0.150i +0.140j +0.100k']
['0.240 -0.030i +0.100j -0.010k', '0.430 -0.240i -0.300j -0.280k']


In [37]:
Vq.inv()

Quaternion-valued array of shape (5, 5):
[[Quaternion(2.750111914327348e-05, -3.652744881739445e-17, -0.3867127945711346, -0.08422602702135787)
  Quaternion(0.11217360841987631, -0.07496143370182133, -0.31302078602278893, 0.0358615930150588)
  Quaternion(0.10760612672799094, -0.07826304730624473, -0.3045356774260821, 0.03649344919376647)
  Quaternion(0.22864208975569894, -0.15276700000967652, -0.2302823128844718, 0.16022951397056762)
  Quaternion(-0.4609411154114941, 0.3111787261287824, 0.08534744781571701, -0.3784303826186013)]
 [Quaternion(-0.612331586945051, -1.1102230246251565e-16, 0.03264649445652912, 0.04050276139200756)
  Quaternion(0.12499110760423511, 0.17089154994669803, 0.14606569507976747, 0.16625651162596672)
  Quaternion(0.12002469207223962, 0.1693947716238076, 0.13896072547528865, 0.16385080560865903)
  Quaternion(0.4557140555100803, -0.22756184618260825, -0.2343839516571944, -0.25632101178126765)
  Quaternion(0.14303579851336415, -0.10810775600200397, -0.104845719146942

In [ ]:
print(Aq)

In [ ]:
print(Aq.complex_adjoint)